In [2]:
include("./constraint_types.jl")
using .ConstraintTypes: LinearConstraints, NoConstraints

using Statistics
using LinearAlgebra
using MLStyle

In [3]:
function unit_hypercube_constraints(d)
    A = [ Matrix(I, d, d); -Matrix(I, d, d) ]
    b = [ ones(d); zeros(d) ]
    return A, b
end
A, b = unit_hypercube_constraints(4)

([1 0 0 0; 0 1 0 0; … ; 0 0 -1 0; 0 0 0 -1], [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0])

In [4]:
constraints = ConstraintTypes.LinearConstraints(A, b)

LinearConstraints([1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; … ; 0.0 0.0 -1.0 0.0; 0.0 0.0 0.0 -1.0], [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0])

# Constraint Builder
As a brief aside, I would like to make it easier to build constraints, with the builder producing a `LinearConstraint` object, along with the specific constraint enforcement mechanism. So ultimately I will have an enforcer and a constraint. There are three general types of constraints:
- Interval: lower and upper bounds
- Ratio: relationships between constraints

The user should start by defining a list of names for their experimental factors.

In [5]:
# Convert to dictionary 
factors = [
    "factor 1",
    "factor 2",
    "factor 3"
]

3-element Vector{String}:
 "factor 1"
 "factor 2"
 "factor 3"

In [6]:
struct Experiment
    factors::Dict{String, Int64}
    constraints::ConstraintTypes.Constraints
    N::Int64 
    K::Int64 
end

In [7]:
function make_experiment(factors::Vector{String}, N::Int64, K::Int64)
    factor_dict = Dict(f => i for (i, f) in enumerate(factors))
    return Experiment(factor_dict, ConstraintTypes.NoConstraints(), N, K)
end

make_experiment (generic function with 1 method)

In [12]:
function with_interval_constraint(exp::Experiment, factor::String, lower::Float64, upper::Float64)
    factor_index = exp.factors[factor]
    lower_bound_row = zeros(exp.K)
    upper_bound_row = zeros(exp.K)
    lower_bound_row[factor_index] = 1
    upper_bound_row[factor_index] = -1

    @match exp.constraints begin
        ConstraintTypes.LinearConstraints(A, b) => begin
            new_A = vcat(A, lower_bound_row', upper_bound_row')
            new_b = vcat(b, lower, -upper)
            new_constraints = ConstraintTypes.LinearConstraints(new_A, new_b)
            Experiment(exp.factors, new_constraints, exp.N, exp.K)
        end

        ConstraintTypes.NoConstraints() => begin
            new_A = vcat(lower_bound_row', upper_bound_row')
            new_b = [lower, -upper]
            new_constraints = ConstraintTypes.LinearConstraints(new_A, new_b)
            Experiment(exp.factors, new_constraints, exp.N, exp.K)
        end

        _ => error("Unsupported constraint type: $(typeof(exp.constraints))")
    end
end

function with_factor_ratio(exp::Experiment, factor_1::String, factor_2::String, ratio::Float64)
    factor1_index = exp.factors[factor_1]
    factor2_index = exp.factors[factor_2]

    row = zeros(exp.K)
    row[factor1_index] = 1
    row[factor2_index] = -ratio

    @match exp.constraints begin
        ConstraintTypes.LinearConstraints(A, b) => begin
            new_A = vcat(A, row')
            new_b = vcat(b, 0.0)
            new_constraints = ConstraintTypes.LinearConstraints(new_A, new_b)
            Experiment(exp.factors, new_constraints, exp.N, exp.K)
        end

        ConstraintTypes.NoConstraints() => begin
            new_A = row'
            new_b = [0.0]
            new_constraints = ConstraintTypes.LinearConstraints(new_A, new_b)
            Experiment(exp.factors, new_constraints, exp.N, exp.K)
        end

        _ => error("Unsupported constraint type: $(typeof(exp.constraints))")
    end
end

function with_linear_constraint(exp::Experiment, constraint::Vector{Float64}, bound::Float64)
    @assert length(constraint) == exp.K "Constraint vector must have length equal to number of factors (K = $(exp.K))"

    @match exp.constraints begin
        ConstraintTypes.LinearConstraints(A, b) => begin
            new_A = vcat(A, constraint')
            new_b = vcat(b, bound)
            new_constraints = ConstraintTypes.LinearConstraints(new_A, new_b)
            Experiment(exp.factors, new_constraints, exp.N, exp.K)
        end

        ConstraintTypes.NoConstraints() => begin
            new_A = constraint'
            new_b = [bound]
            new_constraints = ConstraintTypes.LinearConstraints(new_A, new_b)
            Experiment(exp.factors, new_constraints, exp.N, exp.K)
        end

        _ => error("Unsupported constraint type: $(typeof(exp.constraints))")
    end
end

function with_linear_constraint(exp::Experiment, constraint::Array{Float64, 2}, bound::Vector{Float64})
    @assert size(constraint, 2) == exp.K "Constraint matrix must have K columns (K = $(exp.K))"
    @assert size(constraint, 1) == length(bound) "Number of rows in constraint matrix must match length of bound vector"

    @match exp.constraints begin
        ConstraintTypes.LinearConstraints(A, b) => begin
            new_A = vcat(A, constraint)
            new_b = vcat(b, bound)
            new_constraints = ConstraintTypes.LinearConstraints(new_A, new_b)
            Experiment(exp.factors, new_constraints, exp.N, exp.K)
        end

        ConstraintTypes.NoConstraints() => begin
            new_constraints = ConstraintTypes.LinearConstraints(constraint, bound)
            Experiment(exp.factors, new_constraints, exp.N, exp.K)
        end

        _ => error("Unsupported constraint type: $(typeof(exp.constraints))")
    end
end


with_linear_constraint (generic function with 1 method)

In [13]:
 factors = [
    "factor 1",
    "factor 2",
    "factor 3"
]

exp = make_experiment(factors, 7, 3)
exp = with_interval_constraint(exp, "factor 1", 0.0, 10.0)
exp = with_interval_constraint(exp, "factor 2", 0.5, 1.5)
exp = with_factor_ratio(exp, "factor 1", "factor 2", 2.5)

Experiment(Dict("factor 3" => 3, "factor 1" => 1, "factor 2" => 2), LinearConstraints([1.0 0.0 0.0; -1.0 0.0 0.0; … ; 0.0 -1.0 0.0; 1.0 -2.5 0.0], [0.0, -10.0, 0.5, -1.5, 0.0]), 7, 3)